In [ ]:
import pandas as pd

months = ['07', '08', '09', '10', '11', '12']

회원정보, 신용정보, 승인매출정보, 청구입금정보, 잔액정보, 채널정보, 마케팅정보, 성과정보 = {}, {}, {}, {}, {}, {}, {}, {}

for month in months:
    path_base = f"..\\{{}}\\2018{month}_train_{{}}.parquet"

    회원정보[month] = pd.read_parquet(path_base.format('1.회원정보', '회원정보'))
    신용정보[month] = pd.read_parquet(path_base.format('2.신용정보', '신용정보'))
    승인매출정보[month] = pd.read_parquet(path_base.format('3.승인매출정보', '승인매출정보'))
    청구입금정보[month] = pd.read_parquet(path_base.format('4.청구입금정보', '청구정보'))
    잔액정보[month] = pd.read_parquet(path_base.format('5.잔액정보', '잔액정보'))
    채널정보[month] = pd.read_parquet(path_base.format('6.채널정보', '채널정보'))
    마케팅정보[month] = pd.read_parquet(path_base.format('7.마케팅정보', '마케팅정보'))
    성과정보[month] = pd.read_parquet(path_base.format('8.성과정보', '성과정보'))


In [ ]:

for m in months:
    df = 승인매출정보[m]
    print(f"📌 {m}월 - 이용건수_부분무이자_B0M 고유값:")
    print(df['이용건수_부분무이자_B0M'].unique())
    print(f"📌 {m}월 - 이용금액_부분무이자_B0M 고유값:")
    print(df['이용금액_부분무이자_B0M'].unique())
    print("-" * 60)


📌 07월 - 이용건수_부분무이자_B0M 고유값:
[0]
📌 07월 - 이용금액_부분무이자_B0M 고유값:
[0]
------------------------------------------------------------
📌 08월 - 이용건수_부분무이자_B0M 고유값:
[0]
📌 08월 - 이용금액_부분무이자_B0M 고유값:
[0]
------------------------------------------------------------
📌 09월 - 이용건수_부분무이자_B0M 고유값:
[0]
📌 09월 - 이용금액_부분무이자_B0M 고유값:
[0]
------------------------------------------------------------
📌 10월 - 이용건수_부분무이자_B0M 고유값:
[0]
📌 10월 - 이용금액_부분무이자_B0M 고유값:
[0]
------------------------------------------------------------
📌 11월 - 이용건수_부분무이자_B0M 고유값:
[0]
📌 11월 - 이용금액_부분무이자_B0M 고유값:
[0]
------------------------------------------------------------
📌 12월 - 이용건수_부분무이자_B0M 고유값:
[0]
📌 12월 - 이용금액_부분무이자_B0M 고유값:
[0]
------------------------------------------------------------


In [ ]:
for m in months:
    df = 승인매출정보[m]

    df['총이용건수'] = (
        df['이용건수_신용_B0M'] +
        # df['이용건수_부분무이자_B0M'] +
        df['이용건수_체크_B0M'] +
        df['이용건수_카드론_B0M']
    )

    df['총이용금액'] = (
        df['이용금액_일시불_B0M'] +
        df['이용금액_할부_B0M'] +
        # df['이용금액_부분무이자_B0M'] +
        df['이용금액_CA_B0M'] +
        df['이용금액_체크_B0M'] +
        df['이용금액_카드론_B0M']
    )

    승인매출정보[m] = df  # 변경사항 반영
    print(f"✅ 승인매출정보[{m}] 처리 완료")


✅ 승인매출정보[07] 처리 완료
✅ 승인매출정보[08] 처리 완료
✅ 승인매출정보[09] 처리 완료
✅ 승인매출정보[10] 처리 완료
✅ 승인매출정보[11] 처리 완료
✅ 승인매출정보[12] 처리 완료


In [ ]:
# 회원정보[month] - 기준년월, ID, 연령, 남녀구분코드, Segment, 거주시도명
# 승인매출정보[month] - _1순위업종, 총이용건수, 총이용금액, 최종카드론_대출금액
# 잔액정보[month] - 월평균잔액

In [ ]:
cols = ['기준년월', 'ID', '_1순위업종', '총이용건수', '총이용금액',
        '연령', '남녀구분코드', 'Segment', '거주시도명',
        '월평균잔액', '최종카드론_대출금액']

merged_dfs = {}

for m in months:
    try:
        # 1. 필요한 컬럼 추출
        member = 회원정보[m][['기준년월', 'ID', '연령', '남녀구분코드', 'Segment', '거주시도명']]
        sales = 승인매출정보[m][['기준년월', 'ID', '_1순위업종', '총이용건수', '총이용금액', '최종카드론_대출금액']]

        # ✅ '월중평잔' 컬럼이 있는 경우만 활용하여 이름 변경
        if '월중평잔' in 잔액정보[m].columns:
            balance = 잔액정보[m][['기준년월', 'ID', '월중평잔']].rename(columns={'월중평잔': '월평균잔액'})
        else:
            print(f"⚠ {m}월: '월중평잔' 없음 → 월평균잔액 미포함")
            balance = None

        # 2. 병합
        df = member.merge(sales, on=['기준년월', 'ID'], how='left')
        if balance is not None:
            df = df.merge(balance, on=['기준년월', 'ID'], how='left')

        # 3. 컬럼 정리 (존재하는 컬럼만 유지)
        available_cols = [c for c in cols if c in df.columns]
        df = df[available_cols]

        # 4. 저장
        merged_dfs[m] = df
        print(f"✅ df{m} 병합 완료")

    except Exception as e:
        print(f"❌ {m}월 병합 중 오류: {e}")


✅ df07 병합 완료
✅ df08 병합 완료
✅ df09 병합 완료
✅ df10 병합 완료
✅ df11 병합 완료
✅ df12 병합 완료


In [ ]:
df_all = pd.concat(merged_dfs.values(), ignore_index=True)
print(df_all.shape)
print(df_all.head())

(2400000, 11)
     기준년월            ID _1순위업종  총이용건수  총이용금액   연령  남녀구분코드 Segment 거주시도명  \
0  201807  TRAIN_000000     납부     11  14204  40대       2       D    서울   
1  201807  TRAIN_000001     쇼핑     13   2862  30대       1       E    경기   
2  201807  TRAIN_000002     쇼핑     12  20160  30대       1       C    서울   
3  201807  TRAIN_000003     쇼핑      6  14176  40대       2       D    부산   
4  201807  TRAIN_000004   None     -2      0  40대       2       E    광주   

   월평균잔액  최종카드론_대출금액  
0  17237           0  
1   7967       76723  
2  59917       95094  
3  27854           0  
4      0           0  


In [ ]:
df_all.to_csv('iM_데이콘_유사컬럼.csv')

In [ ]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Data columns (total 11 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   기준년월        int64 
 1   ID          object
 2   _1순위업종      object
 3   총이용건수       int64 
 4   총이용금액       int64 
 5   연령          object
 6   남녀구분코드      int64 
 7   Segment     object
 8   거주시도명       object
 9   월평균잔액       int64 
 10  최종카드론_대출금액  int64 
dtypes: int64(6), object(5)
memory usage: 201.4+ MB


In [ ]:
df_all.isnull().sum()/len(df_all) * 100

기준년월           0.000000
ID             0.000000
_1순위업종        22.499667
총이용건수          0.000000
총이용금액          0.000000
연령             0.000000
남녀구분코드         0.000000
Segment        0.000000
거주시도명          0.000000
월평균잔액          0.000000
최종카드론_대출금액     0.000000
dtype: float64

In [ ]:
df = df_all.dropna()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df.loc[:, '연령'] = le.fit_transform(df['연령'])

In [ ]:
from sklearn.preprocessing import OneHotEncoder
categorical_cols = ['_1순위업종', '거주시도명']
df = pd.get_dummies(df, columns=categorical_cols)

In [ ]:
X = df.drop(columns=['Segment', 'ID', '기준년월'])
y = df['Segment']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)

print("🎯 Accuracy:", accuracy_score(y_val, y_pred))
print("📋 Classification Report:")
print(classification_report(y_val, y_pred))

🎯 Accuracy: 0.8355035725614378
📋 Classification Report:
              precision    recall  f1-score   support

           A       0.80      0.29      0.42       190
           B       0.82      0.31      0.45        29
           C       0.67      0.53      0.59     24257
           D       0.60      0.48      0.53     65411
           E       0.89      0.95      0.91    282115

    accuracy                           0.84    372002
   macro avg       0.76      0.51      0.58    372002
weighted avg       0.82      0.84      0.83    372002



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)

print("🎯 Accuracy:", accuracy_score(y_val, y_pred))
print("📋 Classification Report:")
print(classification_report(y_val, y_pred))

🎯 Accuracy: 0.8356487330713276
📋 Classification Report:
              precision    recall  f1-score   support

           A       0.87      0.21      0.33       190
           B       0.89      0.28      0.42        29
           C       0.67      0.53      0.59     24257
           D       0.61      0.47      0.53     65411
           E       0.88      0.95      0.91    282115

    accuracy                           0.84    372002
   macro avg       0.78      0.49      0.56    372002
weighted avg       0.82      0.84      0.83    372002



In [ ]:
# pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# ✅ SMOTE 적용 (학습 데이터만!)
smote = SMOTE(random_state=42)
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)

print("✅ SMOTE 적용 완료")
print("클래스 분포 (적용 전):")
print(y_train.value_counts())
print("클래스 분포 (적용 후):")
print(pd.Series(y_train_sm).value_counts())

# ✅ 재학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_sm, y_train_sm)

# ✅ 검증
y_pred = model.predict(X_val)

print("🎯 Accuracy:", accuracy_score(y_val, y_pred))
print("📋 Classification Report:")
print(classification_report(y_val, y_pred))


✅ SMOTE 적용 완료
클래스 분포 (적용 전):
Segment
E    1128457
D     261645
C      97029
A        760
B        115
Name: count, dtype: int64
클래스 분포 (적용 후):
Segment
E    1128457
D    1128457
C    1128457
A    1128457
B    1128457
Name: count, dtype: int64
🎯 Accuracy: 0.8328933715410132
📋 Classification Report:
              precision    recall  f1-score   support

           A       0.54      0.34      0.41       190
           B       0.71      0.41      0.52        29
           C       0.63      0.57      0.60     24257
           D       0.59      0.50      0.54     65411
           E       0.89      0.93      0.91    282115

    accuracy                           0.83    372002
   macro avg       0.67      0.55      0.60    372002
weighted avg       0.82      0.83      0.83    372002

